In [35]:
#system
from tqdm import tqdm
from tqdm import tqdm_notebook

#base
import numpy as np
import pandas as pd

#model
import xgboost as xbg
import lightgbm as lgb
import catboost as cbt

#sklearn tools
from sklearn.metrics import f1_score, precision_score, recall_score, precision_recall_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder

#others



import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
train_set = pd.read_csv('input/train_set.csv')
test_set = pd.read_csv('input/test_set.csv')

In [37]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB


In [80]:
test_set.info()
test_set['y'] = -1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         10852 non-null  int64 
 1   age        10852 non-null  int64 
 2   job        10852 non-null  object
 3   marital    10852 non-null  object
 4   education  10852 non-null  object
 5   default    10852 non-null  object
 6   balance    10852 non-null  int64 
 7   housing    10852 non-null  object
 8   loan       10852 non-null  object
 9   contact    10852 non-null  object
 10  day        10852 non-null  int64 
 11  month      10852 non-null  object
 12  duration   10852 non-null  int64 
 13  campaign   10852 non-null  int64 
 14  pdays      10852 non-null  int64 
 15  previous   10852 non-null  int64 
 16  poutcome   10852 non-null  object
 17  y          10852 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 1.5+ MB


In [73]:
train_set.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [74]:
train_set.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [81]:
data = train_set.append(test_set).reset_index(drop=True)
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [82]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID', 'y']]
for i in tqdm_notebook(cat_col):
    data['count_'+i] = data.groupby(i)[i].transform('count')
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))

In [83]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [84]:
feats = [i  for i in data.columns if i not in ['ID', 'y']]
feats

['age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'count_job',
 'count_marital',
 'count_education',
 'count_default',
 'count_housing',
 'count_loan',
 'count_contact',
 'count_month',
 'count_poutcome']

In [85]:
model = lgb.LGBMClassifier(
    boosting_type='GBDT',
    num_leaves=30,
    n_estimators=1500,
    max_depth=-1,
    objective='binary',
    metric = 'auc',
    subsample= 0.95,
    subsample_freq=1,
    colsample_bytree=0.7,
    reg_alpha=0.,
    reg_lambda=0.,
    learning_rate=0.02,
    random_state=2017
)

In [89]:
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']
test_x = data[data['y']==-1][feats]
train_x.shape, test_x.shape

((25317, 25), (10852, 25))

In [98]:
model.fit(train_x, train_y)
test_pre = model.predict_proba(test_x)

array([0.2742011, 0.7257989])

In [105]:
pre = data[data['y']==-1][['ID']]
pre['pred'] = test_pre[:, 1]
test_pre.shape, test_pre[0:5,:]

((10852, 2), array([[0.96517302, 0.03482698],
        [0.9944712 , 0.0055288 ],
        [0.99877612, 0.00122388],
        [0.25366374, 0.74633626],
        [0.9834217 , 0.0165783 ]]))

In [96]:
pre.head()

,ID,pred
25317,25318,0.034827
25318,25319,0.005529
25319,25320,0.001224
25320,25321,0.746336
25321,25322,0.016578


In [114]:
n = 10
kfold = KFold(n_splits=n, shuffle=True, random_state=42)
list(kfold.split(train_x.iloc[1:20]))

[(array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18]),
  array([0, 5])),
 (array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 18]),
  array([ 1, 11])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14, 15, 17, 18]),
  array([ 8, 16])),
 (array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15, 16, 17, 18]),
  array([ 3, 13])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 18]),
  array([15, 17])),
 (array([ 0,  1,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 16, 17, 18]),
  array([2, 9])),
 (array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
  array([ 4, 18])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 13, 14, 15, 16, 17, 18]),
  array([ 7, 12])),
 (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 15, 16, 17, 18]),
  array([10, 14])),
 (array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18]), array([6]))]

In [116]:
res = data[data['y']==-1][['ID']]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    train_x1 = train_x.iloc[train_idx]
    train_y1 = train_y.iloc[train_idx]
    val_x1 = train_x.iloc[val_idx]
    val_y1 = train_y.iloc[val_idx]
    
    model.fit(train_x1,train_y1, eval_set=[(train_x1, train_y1), (val_x1, val_y1)], eval_metric='auc', early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]
    
res['pred'] = res['pred']/kfold.n_splits

[1]	training's auc: 0.890209	valid_1's auc: 0.889424
Training until validation scores don't improve for 100 rounds
[2]	training's auc: 0.907021	valid_1's auc: 0.901357
[3]	training's auc: 0.912792	valid_1's auc: 0.903343
[4]	training's auc: 0.918829	valid_1's auc: 0.907438
[5]	training's auc: 0.927566	valid_1's auc: 0.916754
[6]	training's auc: 0.927007	valid_1's auc: 0.916028
[7]	training's auc: 0.929015	valid_1's auc: 0.916161
[8]	training's auc: 0.929083	valid_1's auc: 0.916183
[9]	training's auc: 0.928682	valid_1's auc: 0.914413
[10]	training's auc: 0.930162	valid_1's auc: 0.915446
[11]	training's auc: 0.930555	valid_1's auc: 0.916745
[12]	training's auc: 0.930514	valid_1's auc: 0.916715
[13]	training's auc: 0.930821	valid_1's auc: 0.917079
[14]	training's auc: 0.931772	valid_1's auc: 0.917735
[15]	training's auc: 0.931979	valid_1's auc: 0.91821
[16]	training's auc: 0.932164	valid_1's auc: 0.918187
[17]	training's auc: 0.932336	valid_1's auc: 0.919179
[18]	training's auc: 0.933709	

[151]	training's auc: 0.950798	valid_1's auc: 0.932299
[152]	training's auc: 0.950823	valid_1's auc: 0.932337
[153]	training's auc: 0.950898	valid_1's auc: 0.932397
[154]	training's auc: 0.951056	valid_1's auc: 0.93243
[155]	training's auc: 0.951147	valid_1's auc: 0.932417
[156]	training's auc: 0.951261	valid_1's auc: 0.93239
[157]	training's auc: 0.951309	valid_1's auc: 0.932352
[158]	training's auc: 0.951359	valid_1's auc: 0.932405
[159]	training's auc: 0.951442	valid_1's auc: 0.932462
[160]	training's auc: 0.951514	valid_1's auc: 0.932486
[161]	training's auc: 0.951616	valid_1's auc: 0.932462
[162]	training's auc: 0.951685	valid_1's auc: 0.932453
[163]	training's auc: 0.951762	valid_1's auc: 0.932393
[164]	training's auc: 0.951792	valid_1's auc: 0.932448
[165]	training's auc: 0.951888	valid_1's auc: 0.932493
[166]	training's auc: 0.951946	valid_1's auc: 0.93249
[167]	training's auc: 0.95198	valid_1's auc: 0.932495
[168]	training's auc: 0.952065	valid_1's auc: 0.932421
[169]	training

[300]	training's auc: 0.961274	valid_1's auc: 0.933303
[301]	training's auc: 0.961342	valid_1's auc: 0.933269
[302]	training's auc: 0.961411	valid_1's auc: 0.933215
[303]	training's auc: 0.961472	valid_1's auc: 0.933356
[304]	training's auc: 0.961546	valid_1's auc: 0.933325
[305]	training's auc: 0.96161	valid_1's auc: 0.933331
[306]	training's auc: 0.961679	valid_1's auc: 0.933347
[307]	training's auc: 0.961753	valid_1's auc: 0.933343
[308]	training's auc: 0.961799	valid_1's auc: 0.933368
[309]	training's auc: 0.961942	valid_1's auc: 0.933334
[310]	training's auc: 0.962005	valid_1's auc: 0.933306
[311]	training's auc: 0.962067	valid_1's auc: 0.933256
[312]	training's auc: 0.962126	valid_1's auc: 0.933269
[313]	training's auc: 0.962232	valid_1's auc: 0.933263
[314]	training's auc: 0.962308	valid_1's auc: 0.933271
[315]	training's auc: 0.9624	valid_1's auc: 0.933322
[316]	training's auc: 0.962481	valid_1's auc: 0.933318
[317]	training's auc: 0.962557	valid_1's auc: 0.933272
[318]	trainin

[449]	training's auc: 0.969716	valid_1's auc: 0.933443
[450]	training's auc: 0.969766	valid_1's auc: 0.933435
[451]	training's auc: 0.969824	valid_1's auc: 0.933434
[452]	training's auc: 0.969893	valid_1's auc: 0.93339
[453]	training's auc: 0.969929	valid_1's auc: 0.933407
[454]	training's auc: 0.969975	valid_1's auc: 0.933434
[455]	training's auc: 0.970035	valid_1's auc: 0.933435
[456]	training's auc: 0.970073	valid_1's auc: 0.933418
[457]	training's auc: 0.970123	valid_1's auc: 0.933421
[458]	training's auc: 0.970191	valid_1's auc: 0.933463
[459]	training's auc: 0.970236	valid_1's auc: 0.933478
[460]	training's auc: 0.970278	valid_1's auc: 0.933471
[461]	training's auc: 0.970339	valid_1's auc: 0.933531
[462]	training's auc: 0.970385	valid_1's auc: 0.933547
[463]	training's auc: 0.970428	valid_1's auc: 0.933565
[464]	training's auc: 0.970473	valid_1's auc: 0.93359
[465]	training's auc: 0.970524	valid_1's auc: 0.933558
[466]	training's auc: 0.970557	valid_1's auc: 0.933563
[467]	traini

[598]	training's auc: 0.976093	valid_1's auc: 0.933743
[599]	training's auc: 0.976128	valid_1's auc: 0.933739
[600]	training's auc: 0.976175	valid_1's auc: 0.933756
[601]	training's auc: 0.976221	valid_1's auc: 0.933786
[602]	training's auc: 0.976251	valid_1's auc: 0.933765
[603]	training's auc: 0.976298	valid_1's auc: 0.933762
[604]	training's auc: 0.976341	valid_1's auc: 0.933755
[605]	training's auc: 0.976382	valid_1's auc: 0.933808
[606]	training's auc: 0.97642	valid_1's auc: 0.93382
[607]	training's auc: 0.976452	valid_1's auc: 0.93382
[608]	training's auc: 0.976489	valid_1's auc: 0.933768
[609]	training's auc: 0.976506	valid_1's auc: 0.933793
[610]	training's auc: 0.976554	valid_1's auc: 0.933798
[611]	training's auc: 0.976573	valid_1's auc: 0.933826
[612]	training's auc: 0.976609	valid_1's auc: 0.933864
[613]	training's auc: 0.976659	valid_1's auc: 0.933859
[614]	training's auc: 0.976707	valid_1's auc: 0.933845
[615]	training's auc: 0.976743	valid_1's auc: 0.933834
[616]	trainin

...

[592]	training's auc: 0.975743	valid_1's auc: 0.939119
[593]	training's auc: 0.975786	valid_1's auc: 0.939107
[594]	training's auc: 0.975811	valid_1's auc: 0.939103
[595]	training's auc: 0.975886	valid_1's auc: 0.939125
[596]	training's auc: 0.975947	valid_1's auc: 0.939101
[597]	training's auc: 0.975987	valid_1's auc: 0.939061
[598]	training's auc: 0.976025	valid_1's auc: 0.939058
[599]	training's auc: 0.976052	valid_1's auc: 0.939051
[600]	training's auc: 0.976078	valid_1's auc: 0.939045
[601]	training's auc: 0.976117	valid_1's auc: 0.939094
[602]	training's auc: 0.976161	valid_1's auc: 0.939082
[603]	training's auc: 0.976197	valid_1's auc: 0.939091
[604]	training's auc: 0.976236	valid_1's auc: 0.939087
[605]	training's auc: 0.976288	valid_1's auc: 0.93908
[606]	training's auc: 0.976316	valid_1's auc: 0.939103
[607]	training's auc: 0.976358	valid_1's auc: 0.939115
[608]	training's auc: 0.976391	valid_1's auc: 0.939079
[609]	training's auc: 0.976426	valid_1's auc: 0.939108
[610]	train

[71]	training's auc: 0.943279	valid_1's auc: 0.919505
[72]	training's auc: 0.943407	valid_1's auc: 0.919564
[73]	training's auc: 0.943502	valid_1's auc: 0.919606
[74]	training's auc: 0.943581	valid_1's auc: 0.91969
[75]	training's auc: 0.943747	valid_1's auc: 0.919555
[76]	training's auc: 0.943841	valid_1's auc: 0.919461
[77]	training's auc: 0.943895	valid_1's auc: 0.919563
[78]	training's auc: 0.944152	valid_1's auc: 0.919734
[79]	training's auc: 0.944268	valid_1's auc: 0.919685
[80]	training's auc: 0.94442	valid_1's auc: 0.919578
[81]	training's auc: 0.944641	valid_1's auc: 0.919531
[82]	training's auc: 0.94476	valid_1's auc: 0.919681
[83]	training's auc: 0.94489	valid_1's auc: 0.919885
[84]	training's auc: 0.945036	valid_1's auc: 0.919901
[85]	training's auc: 0.945163	valid_1's auc: 0.919885
[86]	training's auc: 0.945277	valid_1's auc: 0.919937
[87]	training's auc: 0.945395	valid_1's auc: 0.919803
[88]	training's auc: 0.945499	valid_1's auc: 0.91981
[89]	training's auc: 0.945633	val

[221]	training's auc: 0.956845	valid_1's auc: 0.923306
[222]	training's auc: 0.956927	valid_1's auc: 0.923323
[223]	training's auc: 0.956999	valid_1's auc: 0.923442
[224]	training's auc: 0.957043	valid_1's auc: 0.923426
[225]	training's auc: 0.957103	valid_1's auc: 0.923425
[226]	training's auc: 0.957154	valid_1's auc: 0.923417
[227]	training's auc: 0.957206	valid_1's auc: 0.923415
[228]	training's auc: 0.957265	valid_1's auc: 0.92349
[229]	training's auc: 0.957346	valid_1's auc: 0.923526
[230]	training's auc: 0.957431	valid_1's auc: 0.92355
[231]	training's auc: 0.957492	valid_1's auc: 0.923557
[232]	training's auc: 0.95754	valid_1's auc: 0.923513
[233]	training's auc: 0.957583	valid_1's auc: 0.923478
[234]	training's auc: 0.957636	valid_1's auc: 0.923481
[235]	training's auc: 0.957698	valid_1's auc: 0.92346
[236]	training's auc: 0.957783	valid_1's auc: 0.923481
[237]	training's auc: 0.957852	valid_1's auc: 0.923529
[238]	training's auc: 0.957946	valid_1's auc: 0.923518
[239]	training

[370]	training's auc: 0.966312	valid_1's auc: 0.924603
[371]	training's auc: 0.96634	valid_1's auc: 0.924634
[372]	training's auc: 0.966397	valid_1's auc: 0.924566
[373]	training's auc: 0.966454	valid_1's auc: 0.924578
[374]	training's auc: 0.9665	valid_1's auc: 0.924578
[375]	training's auc: 0.966553	valid_1's auc: 0.924592
[376]	training's auc: 0.96659	valid_1's auc: 0.924575
[377]	training's auc: 0.966636	valid_1's auc: 0.924611
[378]	training's auc: 0.966697	valid_1's auc: 0.924609
[379]	training's auc: 0.966746	valid_1's auc: 0.924558
[380]	training's auc: 0.96679	valid_1's auc: 0.924557
[381]	training's auc: 0.966846	valid_1's auc: 0.924536
[382]	training's auc: 0.966889	valid_1's auc: 0.924505
[383]	training's auc: 0.966944	valid_1's auc: 0.924523
[384]	training's auc: 0.966977	valid_1's auc: 0.924545
[385]	training's auc: 0.967033	valid_1's auc: 0.924596
[386]	training's auc: 0.967088	valid_1's auc: 0.924652
[387]	training's auc: 0.967138	valid_1's auc: 0.92466
[388]	training's

[519]	training's auc: 0.973666	valid_1's auc: 0.925262
[520]	training's auc: 0.973688	valid_1's auc: 0.925241
[521]	training's auc: 0.973742	valid_1's auc: 0.925251
[522]	training's auc: 0.973774	valid_1's auc: 0.925275
[523]	training's auc: 0.973817	valid_1's auc: 0.925252
[524]	training's auc: 0.973865	valid_1's auc: 0.925242
[525]	training's auc: 0.973893	valid_1's auc: 0.92527
[526]	training's auc: 0.973933	valid_1's auc: 0.925234
[527]	training's auc: 0.973985	valid_1's auc: 0.925248
[528]	training's auc: 0.974029	valid_1's auc: 0.925218
[529]	training's auc: 0.974077	valid_1's auc: 0.925207
[530]	training's auc: 0.974114	valid_1's auc: 0.925221
[531]	training's auc: 0.974148	valid_1's auc: 0.925234
[532]	training's auc: 0.974192	valid_1's auc: 0.925259
[533]	training's auc: 0.974233	valid_1's auc: 0.925266
[534]	training's auc: 0.974283	valid_1's auc: 0.925286
[535]	training's auc: 0.974321	valid_1's auc: 0.9253
[536]	training's auc: 0.974363	valid_1's auc: 0.92531
[537]	training